In [1]:
from Crypto.Util.number import *
from random import *
class iSHE:
    
    def __init__(self,k_0 , k_r , k_M):
        self.k_0 = k_0  
        self.k_r = k_r
        self.k_M = k_M 
    def keygen(self):
        #   /**
        #    * k_0 : The bit length of prime numbers p and q is usually 1024
        #    * k_r : Parameter L and the length of the random value are used in the encryption algorithm and are usually 80
        #    * k_M : The message space definition range is usually 30
        #    */
        k_0 = self.k_0
        k_M = self.k_M
        k_r = self.k_r
        assert(k_0 > k_r and k_r > k_M and k_M > 0) # The parameters must satisfy the relationship
        p = getPrime(k_0)
        q = getPrime(k_0)
        N = p * q
        s = randint(0 , N)
        L = getrandbits(k_r)
        self.Mspace = (-2**(k_M-1) , 2**(k_M-1))
        self.sk = (s, p, L)
        self.pk = (k_0,k_r , N , self.Mspace)
        return self.sk, self.pk
    def enc(self,m, d):
        assert(m < self.Mspace[1] and m >= self.Mspace[0])# // Input data must be in space
        tmp = m
        r = getrandbits(self.k_r)#        // r = {0,1}^k_r
        r1 = getrandbits(self.k_0)#       // r' ={0,1}^k_0
        m1 = pow(self.sk[0], d, self.pk[2])#     // m' = s^d
        m1 *= (r * self.sk[2] + tmp)#                    // m' = s*(rL+m)
        m1 = m1 * (1 + r1 * self.sk[1]) % self.pk[2] # // m' = s*(rL+m)*(1+r'p) mod N
        return m1 , d



In [24]:
from sage.all import *

ishe = iSHE(1024 , 600 , 30)
_ , pk = ishe.keygen()
k_0 , k_r , N , _ = pk
cnt = 20
m1list = []
for i in range(cnt):
    m = getrandbits(29)
    m1 , _ = ishe.enc(m , 114514)
    m1list.append(m1)
mbit = k_r * 2
sbit = k_0
M = Matrix(ZZ , cnt+1)
for i in range(cnt):
    M[i , i] = N
    M[-1, i] = m1list[i] 
M[-1 , -1] = 1
res = M.LLL()

In [25]:
q = GCD(N , res[1][-1])
p = N // q
print(isPrime(p))
print(isPrime(q))


True
True


In [18]:
q

175938816068719469857392217067174228977295972057082224256445515916691950307504984015451167797683064353694267329661031310222359441400101748644910609030674464993187453976376114888613796068526836946734159367133912273958329206188788238309707052107350364308070618853207445283139877499630353817401884762625504389983

In [19]:
print(len(bin(p)))

1026
